In [1]:
!pip install timm

In [2]:
import os
import gc
import random
import copy
import cv2
import time

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.transforms as T

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split

import joblib
from tqdm import tqdm
from collections import defaultdict

from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

In [3]:
ROOT_DIR = '../input/cassava-leaf-disease-classification'
TRAIN_DIR = '../input/cassava-leaf-disease-classification/train_images'
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images'

In [4]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(42)

In [5]:
CONFIG = dict(
    seed = 42,
    model_name = 'tf_mobilenetv3_small_100',
    train_batch_size = 32,
    valid_batch_size = 64,
    img_size = 512,
    epochs = 10,
    learning_rate = 1e-4,
    n_accumulate = 1,
    n_fold = 5,
    T_max = 100,
    T_0 = 25,
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    min_lr = 1e-5,
    weight_decay = 1e-6,
    competition = 'Cassava',
    wandb_kernel = 'deb'
)

In [6]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret('wandb_api')
    wandb.login(key=api_key)
    anony=None
except:
    anony = 'must'

In [7]:
def get_train_file_path(image_id):
    return f'{TRAIN_DIR}/{image_id}'
def get_test_file_path(image_id):
    return f'{TEST_DIR}/{image_id}'

df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['file_path'] = df['image_id'].apply(get_train_file_path)

In [8]:
class CassavaDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir = root_dir
        self.df = df
        self.file_name = df['file_path'].values
        self.labels = df['label'].values
        self.transforms = transforms
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_name[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        labels = self.labels[index]
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img, labels

In [9]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

data_augmentations = {
    'train' : A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.HorizontalFlip(),
        A.Normalize(mean=MEAN, std=STD, max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.0),
    
    'valid' : A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(mean=MEAN, std=STD, max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ])
}

In [10]:
class CassavaModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super(CassavaModel, self).__init__()
        self.model = timm.create_model(model_name=model_name, pretrained=pretrained)
        self.in_features = self.model.classifier.in_features
        self.model.reset_classifier(0)
        self.fc = nn.Linear(self.in_features, len(df['label'].unique()))
        self.dropout = nn.Dropout(p=0.6)
        
    def forward(self, images):
        features = self.model(images)
        features = self.dropout(features)
        output = self.fc(features)
        return output

model = CassavaModel(CONFIG['model_name'])
model.to(CONFIG['device'])

In [11]:
def criterion(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets)

In [12]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss = loss / CONFIG['n_accumulate']
            
        scaler.scale(loss).backward()
        
        if (step+1) % CONFIG['n_accumulate'] == 0:
            scaler.step(optimizer)
            scaler.update()
            
            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])
        
    gc.collect()
    
    return epoch_loss

In [19]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    TARGETS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        PREDS.append(preds.view(-1).cpu().detach().numpy())
        TARGETS.append(labels.view(-1).cpu().detach().numpy())
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])
    
    TARGETS = np.concatenate(TARGETS)
    PREDS = np.concatenate(PREDS)
    val_acc = accuracy_score(TARGETS, PREDS)
    gc.collect()
    
    return epoch_loss, val_acc

In [20]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    wandb.watch(model, log_freq=100)
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_acc = 0
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs+1):
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler,
                                           dataloader=train_loader, device=CONFIG['device'], epoch=epoch)
        val_epoch_loss, val_epoch_acc = valid_one_epoch(model, dataloader=valid_loader, 
                                                        device=CONFIG['device'], epoch=epoch)
        
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid Acc'].append(val_epoch_acc)
        
        wandb.log({'Train Loss' : train_epoch_loss})
        wandb.log({'Valid Loss' : val_epoch_loss})
        wandb.log({'Valid Acc' : val_epoch_acc})
        
        print(f'Valid Accuracy : {val_epoch_acc}')
        
        if val_epoch_acc >= best_epoch_acc:
            print(f'{c_}Validation Acc Improved({best_epoch_acc} --> {val_epoch_acc})')
            best_epoch_acc = val_epoch_acc
            run.summary['Best Accuracy'] = best_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f'Epoch{epoch}.bin'
            torch.save(model.state_dict(), PATH)
            wandb.save(PATH)
            print(f'Model Saved{sr_}')
            
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best ACC: {:.4f}".format(best_epoch_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model, history

In [21]:
# DataLoader
def prepare_loader():
    df_train, df_valid = train_test_split(df, test_size=0.3, random_state=CONFIG['seed'], shuffle=True)
    
    train_dataset = CassavaDataset(TRAIN_DIR, df_train, transforms=data_augmentations['train'])
    valid_dataset = CassavaDataset(TRAIN_DIR, df_valid, transforms=data_augmentations['valid'])
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

train_loader, valid_loader = prepare_loader()

In [16]:
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr'])

In [17]:
run = wandb.init(project='Cassava', 
                 config=CONFIG,
                 job_type='Train',
                 anonymous='must')

In [22]:
mode, history = run_training(model, optimizer, scheduler,
                            device=CONFIG['device'], num_epochs=CONFIG['epochs'])

In [23]:
run.finish()

In [48]:
class CassavaTestDataset(Dataset):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir = root_dir
        self.df = df
        self.file_name = df['file_path'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_name[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img

In [42]:
test = pd.DataFrame()
test['file_path'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
test

In [50]:
test_dataset = CassavaTestDataset('../input/cassava-leaf-disease-classification/test_images', test, transforms=data_augmentations['valid'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['valid_batch_size'], num_workers=2, pin_memory=False)

In [51]:
MODEL_PATHS = [
    './Epoch5.bin',
    './Epoch4.bin'
]

In [59]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, images in bar:
        images = images.to(device, dtype=torch.float)
        
        batch_size = images.size
        dataset_size = batch_size
        
        outputs = model(images)
        PREDS.append(torch.softmax(outputs, 1).cpu().detach().numpy())
    
    PREDS = np.concatenate(PREDS, axis=0)
    return PREDS

In [57]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = CassavaModel(CONFIG['model_name'], pretrained=False)
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f'getting predictions for model {i+1}')
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [60]:
preds = inference(MODEL_PATHS, test_loader, CONFIG['device'])

In [63]:
preds

In [62]:
test['label'] = np.argmax(preds, axis=1)
test.head()